In [2]:
import sys
sys.path.append('..//')
sys.path.append('lib/model_gen/')

import merge_consensus as mc
import pod5
#import tracemalloc
import gc
import pandas as pd
import numpy as np
import ast
import pickle
import datetime
import os
import multiprocessing
#tracemalloc.start()
import psutil

In [ ]:
pod5_str = "../../data/large_working_directory/merged_pod5/merged.pod5"
bam_str = "../../data/large_working_directory/rough_consensus_output/large_align.bam"
fasta_str = "../../data/reads_large/230725_PZ_lib_v4_r10/fasta/good.fa"

loading_pool = multiprocessing.Pool(processes=4)
merged_data = loading_pool.starmap(mc.merge_bam_reads_by_id, [(loading_pool.map(mc.load_pod5_data, [pod5_str])[0],
                                                               loading_pool.map(mc.load_in_data, [bam_str])[0],
                                                               loading_pool.map(mc.consensus_formatter, [fasta_str])[0])])
loading_pool.close()
loading_pool.join()

In [ ]:
pod5_str = "../../data/large_working_directory/merged_pod5/merged.pod5"
p5i.load_pod5_data(pod5_str)

pod5 load called...


In [ ]:
pod5_str = "../../data/large_working_directory/merged_pod5/merged.pod5"
bam_str = "../../data/large_working_directory/rough_consensus_output/large_align.bam"
fasta_str = "../../data/reads_large/230725_PZ_lib_v4_r10/fasta/good.fa"

loading_pool = multiprocessing.Pool(processes=4)
merged_data = loading_pool.starmap(mc.merge_bam_reads_by_id, [(loading_pool.map(mc.load_pod5_data, [pod5_str])[0],
                                                               loading_pool.map(mc.load_in_data, [bam_str])[0],
                                                               loading_pool.map(mc.consensus_formatter, [fasta_str])[0])])
loading_pool.close()
loading_pool.join()

pod5


In [4]:
pod5_data[0]

,seq_id,signal,freq
0,0036175b-fc7a-4f57-9c34-5bf6f4862c6a,"[106.97853, 118.42194, 119.46225, 118.94209, 1...",4000
1,009e4f6d-f975-4642-8349-e033500d8018,"[118.59533, 119.80902, 119.288864, 119.80902, ...",4000
2,00ed1d8c-35f1-4fbf-9d0c-77ad74d3428c,"[96.57543, 109.05915, 105.76484, 99.17621, 108...",4000
3,01057a9b-f69d-4acf-8cf6-e376b48504d0,"[124.83719, 124.31703, 119.98241, 118.94209, 1...",4000
4,01487f03-cb51-43ed-b3ec-03001dc0562a,"[129.69197, 117.03486, 119.288864, 117.03486, ...",4000
...,...,...,...
403518,feccb522-b866-4b7b-a37e-1ae119759e72,"[118.24856, 124.83719, 123.27672, 114.260704, ...",4000
403519,ff900236-f0d0-47f1-b6ca-8730f73c3934,"[110.099464, 106.63176, 121.54287, 122.23641, ...",4000
403520,ffcfd044-b23f-49ac-a4ea-dbc9d33b39e3,"[160.90126, 119.635635, 120.15579, 119.98241, ...",4000
403521,ffea84b4-197c-4eef-8cee-eebde83579c7,"[114.43409, 113.047005, 112.180084, 109.75269,...",4000


In [6]:
memory_report()

[ Memory Report 2024-05-14 13:56:56.518338 ]------------------------------------------------
RSS: 4.734124032 gb, VMS: 8.298565632 gb, SHARED: 0.013238272 gb, DATA: 6.079442944 gb
--------------------------------------------------------------------------------------------


In [6]:
unbatched = ['a','b','c','d']
n=3
batched = [unbatched[i:i+n] for i in range(0, len(unbatched), n)]

In [10]:
reads = None

In [7]:
batched

[['a', 'b', 'c'], ['d']]

In [11]:
def save_by_consensus(merged_signal_list_dict, savefile_path):
    '''
    save_by_consensus() saves reads by consensus in their own individual json files.
    
    Parameters:
    merged_signal_list_dict: list of dicts containing relevant data keys: [ref_name,
                                                               quality,
                                                               len,
                                                               ref,
                                                               rev,
                                                               moves,
                                                               sig_len,
                                                               trim_offs,
                                                               freq,
                                                               ref_seq]
    savefile_path: folder that the json files will be exported to, as a str. MAKE SURE IT ENDS WITH '/'.
    
    '''
    
    df = pd.DataFrame(merged_signal_list_dict)
    
    os.makedirs(savefile_path, exist_ok=True)

    consensus_ids = list(df['ref_name'].unique())
    
    n=3
    batched_consensuses = [consensus_ids[i:i+n] for i in range(0, len(consensus_ids), n)]
    
    
    with alive_bar(len(consensus_ids)) as bar:
        
        for batch in batched_consensuses:
            
            pool = multiprocessing.Pool(processes = n)
            pre_assign = functools.partial(save_to_json, df)
            pool.map(pre_assign, batch)
            pool.close()
            for i in range(n):
                bar()
        
def save_to_json(consensus_id,df):
    consensus = df[df['ref_name'] == consensus_id]

    # Condense the data types (Making sure no info is lost)
    consensus.loc[:,'quality']=consensus['quality'].apply(np.int16)
    consensus.loc[:,'len']=consensus['len'].apply(np.int16)
    consensus.loc[:,'ref']=consensus['ref'].apply(np.int16)
    consensus.loc[:,'rev']=consensus['rev'].apply(np.int16)
    consensus.loc[:,'moves']=consensus['moves'].apply(np.asarray)
    consensus.loc[:,'sig_len']=consensus['sig_len'].apply(np.int16)
    consensus.loc[:,'trim_ofs']=consensus['trim_ofs'].apply(np.int16)
    consensus = consensus.reset_index(drop=True)
    # sub method to convert datatypes in a list to a list

    def listconvert(x):
        # submethod listconvert is used to map subsects of data entries to lists.
        for i in range(len(x)):
            x[i]=list(x[i])
        return x

    consensus.loc[:,'signal']=consensus['signal'].apply(listconvert)
    consensus.loc[:,'freq']=consensus['freq'].apply(np.float16)

    # Get the reference name, sequence, and frequency - 
    # #### THIS IS ASSUMING THAT THE FREQUENCY IS THE SAME FOR ALL READS ####
    ref_name = consensus['ref_name'][0]
    ref_seq = consensus['ref_seq'][0]
    freq = consensus['freq'][0]

    # generate the header and filename
    header = {'ref_seq':ref_seq,'freq':freq}
    filename = "cons_merge__{}.json".format(ref_name)

    # generate the filepath
    filepath = savefile_path + filename

    # generate the json object with the header and the dataframe
    json_frame = consensus.to_json() # conver dataframe to json
    json_frame = json.loads(json_frame) # convert json back to dict
    json_out = json.dumps([header, json_frame]) # convert list of dicts to json

    # if the path exists, overwrite it
    if os.path.exists(filepath):
        os.remove(filepath)

    # write the json object
    with open(filepath, 'a') as file:
        file.write(json_out)
    file.close()


In [12]:
import multiprocessing
import functools

def f(x, a):
    return(x*a)
    

if __name__ == '__main__':
    pool = multiprocessing.Pool(processes=30)
    prod_x = functools.partial(f, a=10)
    pool.map(prod_x, np.arange(0, 30, 1))
    pool.close()
    pool.join()

In [13]:
prod_x

functools.partial(<function f at 0x76819cc7e9d0>, a=10)

In [ ]:
len(pd.DataFrame(bam))

In [8]:
fasta_str = "../../data/reads_large/230725_PZ_lib_v4_r10/fasta/good.fa"
fasta = mc.consensus_formatter(fasta_str)

In [ ]:
pd.DataFrame(fasta)

TypeError: cannot pickle 'lib_pod5.pod5_format_pybind.Pod5FileReader' object

In [6]:
import subprocess
def invoke_pod5_importer(path):
    output = subprocess.run(["python", "pod5_importer.py", "{}".format(path)])
    print(output)
    return output.returncode

In [7]:
pod5_df = invoke_pod5_importer(pod5_str)

running...


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)




CompletedProcess(args=['python', 'pod5_importer.py', '../../data/large_working_directory/merged_pod5/merged.pod5'], returncode=1)


In [2]:
df = pd.read_pickle('test.pickle')

KeyboardInterrupt: 

In [5]:
!python pod5_importer.py -h

running...
usage: pod5_importer.py [-h] path

pod5 loading

positional arguments:
  path        Path to Pod5 file.

optional arguments:
  -h, --help  show this help message and exit


In [5]:
def memory_report():
    #snapshot = tracemalloc.take_snapshot()
    #top_stats = snapshot.statistics('lineno')
    proc = psutil.Process(os.getpid())
    rss = proc.memory_info().rss/(1*10**9)
    vms= proc.memory_info().vms/(1*10**9)
    shared = proc.memory_info().shared/(1*10**9)
    data = proc.memory_info().data/(1*10**9)
    #size,peak= tracemalloc.get_traced_memory()
    print("[ Memory Report {} ]------------------------------------------------".format(datetime.datetime.now()))
    print("RSS: {} gb, VMS: {} gb, SHARED: {} gb, DATA: {} gb".format(rss, vms, shared, data))
    #print("Size: {} gb, Peak {} gb".format(size/(1*10**9),peak/(1*10**9)))
    #print("[ Top 5 Memory Uses ]")
    #for stat in top_stats[:5]:
    #    print(stat)
    print("--------------------------------------------------------------------------------------------")

In [4]:
memory_report()

[ Memory Report 2024-05-13 17:38:04.667684 ]------------------------------------------------
RSS: 0.190906368 gb, VMS: 3.750793216 gb, SHARED: 0.07929856 gb, DATA: 1.68767488 gb
--------------------------------------------------------------------------------------------


In [7]:
df = pd.DataFrame(mc.load_pod5_data(pod5_str))

<class 'numpy.ndarray'>


In [1]:
def testt():
    memory_report()
    df2 = pd.read_pickle('export_merge_pod5.pickle')
    print(sys.getsizeof(df2))
    memory_report()
    
    return None

In [5]:
testt()
memory_report()

[ Memory Report 2024-05-13 17:38:07.978003 ]------------------------------------------------
RSS: 0.190906368 gb, VMS: 3.750793216 gb, SHARED: 0.07929856 gb, DATA: 1.68767488 gb
--------------------------------------------------------------------------------------------
89178727
[ Memory Report 2024-05-13 17:38:12.617379 ]------------------------------------------------
RSS: 8.398368768 gb, VMS: 11.957264384 gb, SHARED: 0.080216064 gb, DATA: 9.894146048 gb
--------------------------------------------------------------------------------------------
[ Memory Report 2024-05-13 17:38:12.876167 ]------------------------------------------------
RSS: 4.825284608 gb, VMS: 8.383905792 gb, SHARED: 0.080347136 gb, DATA: 6.320787456 gb
--------------------------------------------------------------------------------------------


In [6]:
size,peak= tracemalloc.get_traced_memory()

(498283, 8168251563)

In [5]:
def tryscale(scale):
    print(scale)
    data_list = []
    for i in range(1):
        data_list = []
        with pod5.Reader(pod5_str) as pod5_file:
            memory_report()
            reads=list(pod5_file.reads())

            for i in range(len(reads[:scale])):
                read = reads[i]
                # save the sequence id, signal, and frequency to a dict
                data_dict = getreaddict(read)
                # append the dict to the list
                data_list.append(data_dict)
            pod5_file.close()
            memory_report()
        memory_report()

In [ ]:
def getreaddict(read):        
    return data_dict
    seq_id = read.read_id
    signal = read.signal_pa
    freq = read.run_info.sample_rate
    data_dict = {'seq_id': str(seq_id),
                 'signal': signal,
                 'freq': freq}

In [3]:
!pip install "dask[dataframe]"

In [9]:
memory_report()

[ Top 5 Memory Uses ]--------------------------
/home/sebastian/.conda/envs/xna_seq/lib/python3.8/tracemalloc.py:65: size=18.7 KiB, count=299, average=64 B
/home/sebastian/.conda/envs/xna_seq/lib/python3.8/tracemalloc.py:532: size=15.5 KiB, count=325, average=49 B
/home/sebastian/.conda/envs/xna_seq/lib/python3.8/site-packages/pod5/reader.py:106: size=13.3 KiB, count=81, average=168 B
/home/sebastian/.conda/envs/xna_seq/lib/python3.8/site-packages/IPython/core/builtin_trap.py:70: size=9248 B, count=1, average=9248 B
/home/sebastian/.conda/envs/xna_seq/lib/python3.8/abc.py:102: size=9217 B, count=95, average=97 B


In [19]:
read_list_dict = []
for read in reads_list:
    seq_id = read.read_id
    signal = read.signal_pa
    freq = read.run_info.sample_rate
    data_dict = {'seq_id': str(seq_id),
                 'signal': signal,
                 'freq': freq}
    read_list_dict.append(data_dict)
memory_report()

RuntimeError: ArrowTableHandle has been closed!